In [1]:
import pandas as pd

In [2]:
qlf = {}

counterYear = 2012
endYear = 2020

while counterYear <= endYear:
  qlf[counterYear] = pd.read_csv(f'raw/{counterYear}.csv')
  counterYear += 1

In [3]:
counterYear = 2012
endYear = 2020

while counterYear <= endYear:
  qlf[counterYear]['Year'] = counterYear
  
  counterYear += 1

In [4]:
frames = []

for key in qlf.keys():
  frames.append(qlf[key])
  
mergedData = pd.concat(frames)

In [14]:
mergedData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606 entries, 0 to 79
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Country                         606 non-null    object 
 1   Quality of Life Index           606 non-null    float64
 2   Purchasing Power Index          606 non-null    float64
 3   Safety Index                    606 non-null    float64
 4   Health Care Index               606 non-null    float64
 5   Cost of Living Index            606 non-null    float64
 6   Property Price to Income Ratio  606 non-null    float64
 7   Traffic Commute Time Index      606 non-null    float64
 8   Pollution Index                 606 non-null    float64
 9   Climate Index                   606 non-null    object 
 10  Year                            606 non-null    int64  
dtypes: float64(8), int64(1), object(2)
memory usage: 73.0+ KB


In [9]:
mergedData.loc[mergedData['Climate Index'] == '-', 'Climate Index'] = -1

In [10]:
mergedData['Climate Index'] = mergedData['Climate Index'].astype('float64')

In [62]:
# a meglévő adatok mellett szükségem lesz még az ISO codeokra, s azt hiszem az ország koordinátákra, de ezt még megkell nézzem a korábbi térképes projektjeimnél, a koordináták csak a térkép esetében fontosak

In [12]:
codes = pd.read_json('raw/codes.json', orient='index')
codes = codes.reset_index()
codes = codes.rename(columns={'index': 'Code', 0: 'Country'})

In [13]:
mergedData.loc[mergedData['Country'] == 'Russia', 'Country'] = 'Russian Federation'
mergedData.loc[mergedData['Country'] == 'United States', 'Country'] = 'United States of America'
mergedData.loc[mergedData['Country'] == 'United Kingdom', 'Country'] = 'United Kingdom of Great Britain'
mergedData.loc[mergedData['Country'] == 'Czech Republic', 'Country'] = 'Czechia'
mergedData.loc[mergedData['Country'] == 'Bosnia And Herzegovina', 'Country'] = 'Bosnia and Herzegovina'
mergedData.loc[mergedData['Country'] == 'South Korea', 'Country'] = 'Korea, Republic of'
mergedData.loc[mergedData['Country'] == 'Vietnam', 'Country'] = 'Viet Nam'
mergedData.loc[mergedData['Country'] == 'Iran', 'Country'] = 'Iran (Islamic Republic of)'
mergedData.loc[mergedData['Country'] == 'Moldova', 'Country'] = 'Moldova, Republic of'
mergedData.loc[mergedData['Country'] == 'Venezuela', 'Country'] = 'Venezuela (Bolivarian Republic of)'
mergedData.loc[mergedData['Country'] == 'Bolivia', 'Country'] = 'Bolivia (Plurinational State of)'

In [14]:
mergedData = mergedData.merge(codes, how='left', left_on='Country', right_on='Country')

In [77]:
mergedData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606 entries, 0 to 605
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Country                         606 non-null    object 
 1   Quality of Life Index           606 non-null    float64
 2   Purchasing Power Index          606 non-null    float64
 3   Safety Index                    606 non-null    float64
 4   Health Care Index               606 non-null    float64
 5   Cost of Living Index            606 non-null    float64
 6   Property Price to Income Ratio  606 non-null    float64
 7   Traffic Commute Time Index      606 non-null    float64
 8   Pollution Index                 606 non-null    float64
 9   Climate Index                   606 non-null    float64
 10  Year                            606 non-null    int64  
 11  Code                            606 non-null    object 
dtypes: float64(9), int64(1), object(2)
m

In [15]:
mergedData.loc[mergedData['Code'].isnull(), 'Country']

Series([], Name: Country, dtype: object)

In [79]:
mergedData.to_csv('raw/mergedButRaw.csv', index = False)

In [18]:
gdp = pd.read_csv('raw/gdp.csv')
gdp.drop(labels=list(map(lambda x : str(x), range(1960, 2011))), axis=1, inplace=True)
gdp.drop(labels=['Country Name'], axis=1, inplace=True)

In [20]:
gdp = gdp.melt(id_vars=['Country Code'], value_vars=list(map(lambda x : str(x), range(2011, 2021))), var_name='Year', value_name='GDP').sort_values(['Country Code', 'Year'])

In [18]:
gdp = gdp.round({'GDP': 2})
gdp['Year'] = gdp['Year'].astype('int64')

In [19]:
gdp = gdp.merge(mergedData, how='right', left_on=['Country Code', 'Year'], right_on=['Code', 'Year'])

In [20]:
gdp.drop(labels=['Country Code'], axis=1, inplace=True)

In [21]:
normalizedData = gdp

In [151]:
normalizedData.to_csv('normalized/normalized.csv', index=False)
normalizedData.to_json('normalized/normalized.json', orient='records')

In [22]:
hunISO = pd.read_csv('raw/hungarianISO.csv')
hunISO

,Ország / Régió,Numerikus,Alpha-3,Alpha-2,Helyi ISO kódok
0,Afganisztán,4,AFG,AF,ISO 3166-2:AF
1,Åland,248,ALA,AX,ISO 3166-2:AX
2,Albánia,8,ALB,AL,ISO 3166-2:AL
3,Algéria,12,DZA,DZ,ISO 3166-2:DZ
4,Amerikai Csendes-óceáni szigetek,581,UMI,UM,ISO 3166-2:UM
...,...,...,...,...,...
246,Vietnám,704,VNM,VN,ISO 3166-2:VN
247,Wallis és Futuna,876,WLF,WF,ISO 3166-2:WF
248,Zambia,894,ZMB,ZM,ISO 3166-2:ZM
249,Zimbabwe,716,ZWE,ZW,ISO 3166-2:ZW


In [23]:
hunISO.drop(labels=['Helyi ISO kódok', 'Alpha-2', 'Numerikus'], axis=1, inplace=True)

In [24]:
hunISO.rename(columns={'Ország / Régió': 'Country'}, inplace=True)

In [25]:
hunISO.rename(columns={'Alpha-3': 'Code'}, inplace=True)

In [26]:
merged = normalizedData.merge(hunISO, how='left', left_on='Code', right_on='Code')

In [28]:
merged.rename(columns={'Country_x': 'Country_EN', 'Country_y': 'Country_HU'}, inplace=True)

In [30]:
merged.to_csv('normalized/normalized.csv', index=False)
merged.to_json('normalized/normalized.json', orient='records')